# API Mercado libre (ML_api)

El objetivo es poder armar un dataset con registros de inmuebles publicados en la página de mercado libre.
Los datos de clasidicados son públicos, detallan precios, condiciones de venta/alquiler, características y zonificación del inmueble.
Estos datos publicados por Mercado Libre (ML) en su página http://www.mercadolibre.com.ar son registrados por propietarios o agentes inmobiliarios para su libre consulta.


Para mayor detalle inspeccionar la pág oficial de ML.

In [2]:
# Imports necesarios
import requests
from bs4 import BeautifulSoup
import re
from lxml import etree
import pandas as pd
import json
from pprint import pprint
import numpy as np
import time

### API Mercado libre: _propiedades_.

Vamos a hacer peticiones (GET) a la api pública de mercado libre.

La búsqueda puede ser performada por query (consulta global) o category (dirigida a una categoría):

Performs a search by query

    GET
        /sites/MLA/search?q=

Performs a search by category

    GET
        /sites/MLA/search?category=

Performs a search by seller_id

    GET
        /sites/MLA/search?seller_id=

Performs a search by nickname

    GET
        /sites/MLA/search?nickname=

Performs a search by special filters

    GET
        /sites/MLA/search?special_filter=

Performs a search with a specific sorting method

    GET
        /sites/MLA/search?q=ipod&sort=sortId=

Performs a search applying filters

    GET
        /sites/MLA/search?q=ipod&FilterID=FilterValue

En el header tiene los siguientes parámetros {code} que podemos modificar

Utilizaremos el siguiente métos pero inspeccionaremos el funcionamiento de otras búsquedas.

url:

    https://api.mercadolibre.com/sites/MLA/search?category={code}&_PublishedToday_{code}_&limit={code}&offset={code}

Parámetros por categoría:

    **search?category**
    Tipo de propiedades (in la terminación luego de _ aplica a toda operación 'venta', 'alquiler', etc.)
    casas = 'MLA1466' #_242075_242060'
    campos = 'MLA1496_242059'
    cocheras = 'MLA50541_267198'
    departamentos = 'MLA1472_242062'
    depositos_galpones = 'MLA1475_245003'
    locales = 'MLA79242_242065'
    oficinas = 'MLA50538_242067'
    phs = 'MLA105179_242069'
    quintas = 'MLA50547_242070'
    terrenos_lotes = 'MLA1493_242071'

Tiempo de publicación:

    **_PublishedToday_** 
    ('YES', 'NO')
    
paginación

    **limit**
    máx = 50

    **offset**
    número de paginación
    0, 1 , ..., n

códigos de operación

    venta = 242075, alquiler = 242073, alquiler_temporal = 242074

# Propiedades de interes.

In [3]:
# Hay más, estas son las de interes por category id
property_type = {
    'departamentos': 'MLA1472',
    'casas': 'MLA1466',
    'terrenos_lotes': 'MLA1493',
    'phs': 'MLA105179',
    'locales': 'MLA79242',
    'oficinas': 'MLA50538',
    'depositos_galpones': 'MLA1475',
    'cocheras': 'MLA50541',
    'campos': 'MLA1496',
    'quintas': 'MLA50547',
}

# Estuctura de la respuesta a la API

### Data response:

    {
      "site_id": "MLA",
      "country_default_time_zone": "GMT-03:00",
      "paging": {},
      "results": [],
      "sort": {},
      "available_sorts": [],
      "filters": [],
      "available_filters": []
    }

Referencia al citio y la zona.
    
    "site_id": "MLA"
    "country_default_time_zone": "GMT-03:00"

Información de la búsqueda y paginación de la request.
    
    "paging": {
        "total": 219007,            # Total de registros para búsqueda.
        "primary_results": 1000,    # Registros relevantes?
        "offset": 0,                # Inico de búsqueda del [0 ; total/limit) devuelve "_limit_" registros
        "limit": 50,                # Registros devueltos (máx 50) por petición

Informacion requerida.

    "results": []                   # len("results") = limit, siempre que no sea la última página.

# Inspección de prueba

hacemos una petición de un solo registro para conocer la estructura de la información.

In [4]:
# Prueba
#: params
today = 'YES'
# Parámetros mínimos para inspección de estructura.
limit = 3
offs = 0

In [5]:
# url = "https://api.mercadolibre.com/sites/MLA/search?q=Departamentos&date_from=2021-01-01&date_to=2022-02-01&limit=3"
# r = requests.get(url)
# r.status_code

In [6]:
#'https://api.mercadolibre.com/sites/MLA/search?category=¿?&_PublishedToday_¿?limit=¿?&offset=¿?' # f'search?q=Departamentos'\
api = f'https://api.mercadolibre.com/sites/MLA/'\
        f'search?category={property_type["departamentos"]}'\
        f'&_PublishedToday_{today}'\
        f'&limit={limit}'\
        f'&offset={offs}'

r = requests.get(api)
r.status_code


200

## Observando la respuesta

Buscamos ver como se estructuran los datos para resgistrar solo aquellos que puedan ser de interes.

In [7]:
data = json.loads(r.content)

type(data), len(data), data.keys()

(dict,
 8,
 dict_keys(['site_id', 'country_default_time_zone', 'paging', 'results', 'sort', 'available_sorts', 'filters', 'available_filters']))

Vemos que el json de respuesta contiene 8 entradas.
Veamos que información trae cada una de ellas.

In [8]:
for i, key in enumerate(data):
    # result and avilable_filters: inspección por separado.
    if key != 'results' and key != 'available_filters':
        print(i ,'-> key: ',key)
        pprint(data[key])
        print(40*'-')


0 -> key:  site_id
'MLA'
----------------------------------------
1 -> key:  country_default_time_zone
'GMT-03:00'
----------------------------------------
2 -> key:  paging
{'limit': 3, 'offset': 0, 'primary_results': 1000, 'total': 277984}
----------------------------------------
4 -> key:  sort
{'id': 'relevance', 'name': 'Más relevantes'}
----------------------------------------
5 -> key:  available_sorts
[{'id': 'price_asc', 'name': 'Menor precio'},
 {'id': 'price_desc', 'name': 'Mayor precio'}]
----------------------------------------
6 -> key:  filters
[{'id': 'category',
  'name': 'Categorías',
  'type': 'text',
  'values': [{'id': 'MLA1472',
              'name': 'Departamentos',
              'path_from_root': [{'id': 'MLA1459', 'name': 'Inmuebles'},
                                 {'id': 'MLA1472', 'name': 'Departamentos'}]}]}]
----------------------------------------


## 'available_filters'

In [9]:
type(data['available_filters']), len(data['available_filters'])

(list, 32)

aviable_filter contiene 32 variables.

Diccionarios con 4 llaves. Contiene los filtros hablitados en la variable filter.

    ['id', 'name', 'type', 'values']


In [10]:
# Inspección de 'id', 'name', 'type', 'values'.
for i, ty in enumerate (data['available_filters']):
    print(f'{i} -> id = {ty["id"]}:\n{ty["name"]}[{ty["type"]}]: {ty["values"]}', end="\n")
    print(50*'-')

0 -> id = official_store:
Tiendas oficiales[text]: [{'id': 'all', 'name': 'Todas las tiendas oficiales', 'results': 36617}, {'id': '2973', 'name': 'Tormes Propiedades', 'results': 254}, {'id': '2810', 'name': 'Miranda Bosch', 'results': 568}, {'id': '2695', 'name': 'Goldstein Propiedades', 'results': 676}, {'id': '2743', 'name': 'Sistema Coldwell Banker', 'results': 1961}, {'id': '2777', 'name': 'Sarro Pucheta Propiedades', 'results': 505}, {'id': '3013', 'name': 'Gilges Inmobiliaria', 'results': 281}, {'id': '2969', 'name': 'Turdo Estudio Inmobiliario', 'results': 57}]
--------------------------------------------------
1 -> id = state:
Ubicación[text]: [{'id': 'TUxBUENBUGw3M2E1', 'name': 'Capital Federal', 'results': 83535}, {'id': 'TUxBUENPU2ExMmFkMw', 'name': 'Bs.As. Costa Atlántica', 'results': 34486}, {'id': 'TUxBUEdSQWU4ZDkz', 'name': 'Bs.As. G.B.A. Norte', 'results': 33156}, {'id': 'TUxBUFNBTmU5Nzk2', 'name': 'Santa Fe', 'results': 28672}, {'id': 'TUxBUEdSQWVmNTVm', 'name': 'Bs.

#### Resumen _'_available_filter_'_

Para atomizar el pedido a la API de ML se puede aplicar diferentes filtros concatenados utlizando
* Grupo de la búsqueda:
    /search?q=id o /search?category=id 
* Filtros concatendados:
    &FilterID=FilterValue&FilterID=FilterValue  

ejemplo:
URI:
'https://api.mercadolibre.com/sites/MLA/'

| Búsqueda |  q_id_filter| category_id_filter|
| --- | --- | --- |
| propiedades = Departamento | /search?q=Departamentos | /search?category=MLA1472|

URI:
'https://api.mercadolibre.com/sites/MLA/search?category=MLA1472'

| Filtro | filter_key | value | syntax |
| --- | --- | --- | --- |
| Antigüedad = 6 a 25 años | PROPERTY_AGE | [6años-25años] | &PROPERTY_AGE=[6años-25años] |
| Ambientes = 3 y 4 | ROOMS | [3-4] | &ROOMS=[3-4] |
| Ubicación = Capital Federal | state | TUxBUENBUGw3M2E1 | &state=TUxBUENBUGw3M2E1 |


URI:
'https://api.mercadolibre.com/sites/MLA/search?category=MLA1472&PROPERTY_AGE=[6años-25años]&ROOMS=[3-4]&state=TUxBUENBUGw3M2E1'


## Inspección de 'results'

In [11]:
# # results
len(data['results']), type(data['results'])

(3, list)

Es una lista con un solo contenido.
Veamos que tiene un dict con 41 varialbes.

In [12]:
# results, largo, tipo y contenidos.
len(data['results'][0]), type(data['results'][0]), data['results'][0].keys()

(41,
 dict,
 dict_keys(['id', 'site_id', 'title', 'seller', 'price', 'prices', 'sale_price', 'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode', 'listing_type_id', 'stop_time', 'condition', 'permalink', 'thumbnail', 'thumbnail_id', 'accepts_mercadopago', 'installments', 'address', 'promotions', 'shipping', 'seller_address', 'seller_contact', 'location', 'attributes', 'original_price', 'category_id', 'official_store_id', 'domain_id', 'catalog_product_id', 'tags', 'order_backend', 'use_thumbnail_id', 'offer_score', 'offer_share', 'match_score', 'winner_item_id', 'melicoin', 'discounts', 'inventory_id']))

Conte nidos de results[0].

In [13]:
# results
for i, k in enumerate (data['results'][0]):
    if k != 'attributes':
        print(f'{i} -> id = {k}:\nValue = {data["results"][0][k]}')#: {ty["title"]}', end="\n")
        print(50*'-')

0 -> id = id:
Value = MLA1229495352
--------------------------------------------------
1 -> id = site_id:
Value = MLA
--------------------------------------------------
2 -> id = title:
Value = Depto Muy Luminoso - 2 Ambientes 
--------------------------------------------------
3 -> id = seller:
Value = {'id': 42642604, 'permalink': 'http://perfil.mercadolibre.com.ar/VIENTODECAMBIO2', 'registration_date': '2011-01-21T11:14:19.000-04:00', 'car_dealer': False, 'real_estate_agency': False, 'tags': ['normal', 'credits_profile', 'messages_as_seller'], 'seller_reputation': {'power_seller_status': None, 'level_id': None, 'metrics': {'cancellations': {'period': '60 months', 'rate': 0, 'value': 0}, 'claims': {'period': '60 months', 'rate': 0, 'value': 0}, 'delayed_handling_time': {'period': '60 months', 'rate': 0, 'value': 0}, 'sales': {'period': '60 months', 'completed': 0}}, 'transactions': {'canceled': 0, 'period': 'historic', 'total': 0, 'ratings': {'negative': 0, 'neutral': 0, 'positive': 

### Vemos que hay dentro de '_results_' '_attributes_'

In [14]:
# keys.
list(data['results'][0]['attributes'][0].keys())

['attribute_group_name',
 'value_type',
 'id',
 'name',
 'value_id',
 'value_name',
 'value_struct',
 'values',
 'attribute_group_id',
 'source']

In [15]:
for i, attr in enumerate(data['results'][0]['attributes']):
    print(f'{i} -> id = {attr["id"]}:\n')
    pprint(attr)
    print(50*'-')

0 -> id = HAS_AIR_CONDITIONING:

{'attribute_group_id': 'COMOYAMEN',
 'attribute_group_name': 'Comodidades y amenities',
 'id': 'HAS_AIR_CONDITIONING',
 'name': 'Aire acondicionado',
 'source': 6316276763983939,
 'value_id': '242085',
 'value_name': 'Sí',
 'value_struct': None,
 'value_type': 'boolean',
 'values': [{'id': '242085',
             'name': 'Sí',
             'source': 6316276763983939,
             'struct': None}]}
--------------------------------------------------
1 -> id = BEDROOMS:

{'attribute_group_id': 'FIND',
 'attribute_group_name': 'Ficha técnica',
 'id': 'BEDROOMS',
 'name': 'Dormitorios',
 'source': 6316276763983939,
 'value_id': None,
 'value_name': '1',
 'value_struct': None,
 'value_type': 'number',
 'values': [{'id': None,
             'name': '1',
             'source': 6316276763983939,
             'struct': None}]}
--------------------------------------------------
2 -> id = COVERED_AREA:

{'attribute_group_id': 'FIND',
 'attribute_group_name': 'Ficha t

In [16]:
data['results'][0]['seller']['real_estate_agency']

False

# Buscando los datos de interes

 id     | header | type | description | location|
 :----: | :----: | :----: | :---- | :----
 0  | id           | [str] | Identificador de la publicación, permite .../items/id_items| data['results'][0]['id']
 1  | star_date    | [dtime] | Inicio de la publicación, solo puedo acceder si ingreso a cada publicación| .../items/id_items
 2  | title        | [str] | Título de la publicación                             | data['results'][0]['title']
 3  | condition    | [str] | Describe si el inmueble es nuevo o usado             | data['results'][0]['condition']
 4  | price        | [float] | Importe publicado del tipo de operación            | data['results'][0]['price']
 5  | end_date     | [dtime] | Fecha en la que se programa el fin de la publicación| data['results'][0]['stop_time']
 6  | currency     | [str] | Típo de moneda                                       | data['results'][0]['currency_id']
 7  | seller_type  | [bool] | Publicado por agente imobiliario o tiendas oficiales| data['results'][0]['seller']['real_estate_agency']
 8  | country      | [float] | Identificador del pasí ej. "AR" Argentina          | data['results'][0]['location']['country][name']
 9 | state        | [str] | Región, zona, localidades ej. Bs.As. G.B.A. Oeste    | data['results'][0]['location']['state']['name']
 10 | city         | [str] | Ciudad, Partido ej. La Matanza, Capital Federal      | data['results'][0]['location']['city']['name']
 11 | neighborhood | [str] | Localidad, Barrio, vecindario ej. Ramos Mejía, Belgrano| data['results'][0]['location']['neighborhood']['name']
 12 | lat          | [float] | Latitud                                            | data['results'][0]['location']['latitude']
 13 | lon          | [float] | Longitud                                           | data['results'][0]['location']['longitude']
 14 | last_update  | [dtime]| Última actualización del precio publicado           | data['results'][0]['prices']['prices']['last_updated']
 15 | property_type| [str] | Departamento, casa, PH, casa quinta                  | data['results'][0]['attributes'][{'id':'PROPERTY_TYPE', 'value_name': "n"}]
 16 | rooms        | [int] | Cantidad de ambientes                                | data['results'][0]['attributes'][{'id':'ROOMS', 'value_name': 'n'}]
 17 | bathrooms    | [int] | Cantidad de baños                                    | data['results'][0]['attributes'][{'id':'FULL_BATHROOMS', 'value_name': 'n'}]
 18 | bedrooms     | [int] | Cantidad de dormitorios                              | data['results'][0]['attributes'][{'id':'BEDROOMS', 'value_name': "n"}]
 19 | operation_type | [str] | Alquiler, alquiler temporal, venta                 | data['results'][0]['attributes'][{'id':'OPERATION', 'value_name': "n"}]
 20 | surface_covered | [float] | Supeficie cubierta en m²                        | data['results'][0]['attributes'][{'id':'COVERED_AREA', 'value_name': "n"}]
 21 | surface_tota | [float] | Supeficie total en m²                              | data['results'][0]['attributes'][{'id':'TOTAL_AREA', 'value_name': "n"}]
   

### Estructurando los datos.

La respuesta de la api viene con una estructura que debemos "aplanar" para conformar un dataframe de pandas

    r = requests.get(api)

Para ello utilizaremos dos estrategias utilizando el normalizador de pandas
    
    pd.json_normalize(data, redord_path, meta)

 tomando com input (1, 2):

        data = json.loads(r.content)

1. Utilizaremos en el parámetro *redor_path* la llave _results_ que viene el la respuesta. Para extraer parte la información buscada.
    
        jsn = r.json()

2. El dict _jsn_ nos permitirá acceder a mayor "profundidad" en el json y estructurar el contenido de las claves _prices.prices_ y _attributes_, 
exploramos  el parámetro *meta* pero la estructura de respuesta de la api no permite aplanar el json en un solo paso, para ello utilizamos este método.

In [17]:
data_col = [
    'id', 'title', 'condition', 'price', 'stop_time', 'currency_id',
    'seller_real_estate_agency',
    'location_country_name', 'location_state_name', 'location_city_name',
    'location_neighborhood_name', 'location_latitude','location_longitude',
    ]
jsnPPcol = [
    'm_id', 'last_updated',
    ]
jsnATTcol = [
    'PROPERTY_TYPE', 'ROOMS', 'FULL_BATHROOMS',
     'BEDROOMS', 'OPERATION', 'COVERED_AREA', 'TOTAL_AREA',
    ]

In [18]:
dd = pd.json_normalize(data, record_path=['results'], sep='_')[data_col]

In [19]:
dd

,id,title,condition,price,stop_time,currency_id,seller_real_estate_agency,location_country_name,location_state_name,location_city_name,location_neighborhood_name,location_latitude,location_longitude
0,MLA1229495352,Depto Muy Luminoso - 2 Ambientes,used,65000,2023-01-06T04:01:42.000Z,ARS,False,Argentina,Capital Federal,Capital Federal,Villa Urquiza,-34.574127,-58.484340
1,MLA1227217684,Alquiler Departamentos 2 Ambientes Villa Gesel...,used,7800,2023-11-06T04:01:14.000Z,ARS,False,Argentina,Bs.As. Costa Atlántica,Villa Gesell,Zona Sur,-37.279080,-56.984270
2,MLA1228990003,Departamento - Villa Crespo,used,95000,2022-12-11T20:19:56.000Z,ARS,True,Argentina,Capital Federal,Capital Federal,Villa Crespo,-34.601560,-58.433075


In [20]:
# Formateamos la request a un json.
jsn = r.json()

In [21]:
dp = pd.json_normalize(jsn['results'],
     ['prices', 'prices'], meta='id', meta_prefix='m_')[jsnPPcol]

In [22]:
dp

,m_id,last_updated
0,MLA1229495352,2022-11-07T23:42:54Z
1,MLA1227217684,2022-11-12T14:34:40Z
2,MLA1228990003,2022-11-07T14:24:10Z


In [23]:
da = pd.json_normalize(jsn['results'],
     'attributes', meta='id', meta_prefix='m_')

da = da.loc[
        da.id.isin(jsnATTcol),
        ['m_id', 'id', 'value_name']
    ].pivot(index='m_id', columns='id', values='value_name')

da.reset_index(inplace=True)

In [24]:
da

id,m_id,BEDROOMS,COVERED_AREA,FULL_BATHROOMS,OPERATION,PROPERTY_TYPE,ROOMS,TOTAL_AREA
0,MLA1227217684,1,33 m²,1,Alquiler temporal,Departamento,2,38 m²
1,MLA1228990003,2,48 m²,2,Alquiler,Departamento,3,51 m²
2,MLA1229495352,1,35 m²,1,Alquiler,Departamento,2,35 m²


In [25]:
df1 = dd.merge(da, left_on='id', right_on='m_id', how='left')


In [26]:
df1 = df1.merge(dp, left_on='id', right_on='m_id', how='left')

df1.drop(columns=['m_id_x', 'm_id_y'], inplace=True)

In [27]:
df1

,id,title,condition,price,stop_time,currency_id,seller_real_estate_agency,location_country_name,location_state_name,location_city_name,...,location_latitude,location_longitude,BEDROOMS,COVERED_AREA,FULL_BATHROOMS,OPERATION,PROPERTY_TYPE,ROOMS,TOTAL_AREA,last_updated
0,MLA1229495352,Depto Muy Luminoso - 2 Ambientes,used,65000,2023-01-06T04:01:42.000Z,ARS,False,Argentina,Capital Federal,Capital Federal,...,-34.574127,-58.484340,1,35 m²,1,Alquiler,Departamento,2,35 m²,2022-11-07T23:42:54Z
1,MLA1227217684,Alquiler Departamentos 2 Ambientes Villa Gesel...,used,7800,2023-11-06T04:01:14.000Z,ARS,False,Argentina,Bs.As. Costa Atlántica,Villa Gesell,...,-37.279080,-56.984270,1,33 m²,1,Alquiler temporal,Departamento,2,38 m²,2022-11-12T14:34:40Z
2,MLA1228990003,Departamento - Villa Crespo,used,95000,2022-12-11T20:19:56.000Z,ARS,True,Argentina,Capital Federal,Capital Federal,...,-34.601560,-58.433075,2,48 m²,2,Alquiler,Departamento,3,51 m²,2022-11-07T14:24:10Z


In [28]:
df2 = pd.DataFrame(columns=data_col + jsnPPcol+ jsnATTcol).drop(columns=['m_id'])
df2.columns

Index(['id', 'title', 'condition', 'price', 'stop_time', 'currency_id',
       'seller_real_estate_agency', 'location_country_name',
       'location_state_name', 'location_city_name',
       'location_neighborhood_name', 'location_latitude', 'location_longitude',
       'last_updated', 'PROPERTY_TYPE', 'ROOMS', 'FULL_BATHROOMS', 'BEDROOMS',
       'OPERATION', 'COVERED_AREA', 'TOTAL_AREA'],
      dtype='object')

In [29]:
# df2.append(df1, ignore_index=True)
pd.concat([df2,df1], ignore_index=True)

,id,title,condition,price,stop_time,currency_id,seller_real_estate_agency,location_country_name,location_state_name,location_city_name,...,location_latitude,location_longitude,last_updated,PROPERTY_TYPE,ROOMS,FULL_BATHROOMS,BEDROOMS,OPERATION,COVERED_AREA,TOTAL_AREA
0,MLA1229495352,Depto Muy Luminoso - 2 Ambientes,used,65000,2023-01-06T04:01:42.000Z,ARS,False,Argentina,Capital Federal,Capital Federal,...,-34.574127,-58.484340,2022-11-07T23:42:54Z,Departamento,2,1,1,Alquiler,35 m²,35 m²
1,MLA1227217684,Alquiler Departamentos 2 Ambientes Villa Gesel...,used,7800,2023-11-06T04:01:14.000Z,ARS,False,Argentina,Bs.As. Costa Atlántica,Villa Gesell,...,-37.279080,-56.984270,2022-11-12T14:34:40Z,Departamento,2,1,1,Alquiler temporal,33 m²,38 m²
2,MLA1228990003,Departamento - Villa Crespo,used,95000,2022-12-11T20:19:56.000Z,ARS,True,Argentina,Capital Federal,Capital Federal,...,-34.601560,-58.433075,2022-11-07T14:24:10Z,Departamento,3,2,2,Alquiler,48 m²,51 m²


In [30]:
df1.reset_index(inplace=True)

In [31]:
df2 = df1.copy()

In [32]:
# Juntamos dos respuestas simuladas
df1 = pd.concat([df1,df2], ignore_index=True).drop(columns='index')
df1

,id,title,condition,price,stop_time,currency_id,seller_real_estate_agency,location_country_name,location_state_name,location_city_name,...,location_latitude,location_longitude,BEDROOMS,COVERED_AREA,FULL_BATHROOMS,OPERATION,PROPERTY_TYPE,ROOMS,TOTAL_AREA,last_updated
0,MLA1229495352,Depto Muy Luminoso - 2 Ambientes,used,65000,2023-01-06T04:01:42.000Z,ARS,False,Argentina,Capital Federal,Capital Federal,...,-34.574127,-58.484340,1,35 m²,1,Alquiler,Departamento,2,35 m²,2022-11-07T23:42:54Z
1,MLA1227217684,Alquiler Departamentos 2 Ambientes Villa Gesel...,used,7800,2023-11-06T04:01:14.000Z,ARS,False,Argentina,Bs.As. Costa Atlántica,Villa Gesell,...,-37.279080,-56.984270,1,33 m²,1,Alquiler temporal,Departamento,2,38 m²,2022-11-12T14:34:40Z
2,MLA1228990003,Departamento - Villa Crespo,used,95000,2022-12-11T20:19:56.000Z,ARS,True,Argentina,Capital Federal,Capital Federal,...,-34.601560,-58.433075,2,48 m²,2,Alquiler,Departamento,3,51 m²,2022-11-07T14:24:10Z
3,MLA1229495352,Depto Muy Luminoso - 2 Ambientes,used,65000,2023-01-06T04:01:42.000Z,ARS,False,Argentina,Capital Federal,Capital Federal,...,-34.574127,-58.484340,1,35 m²,1,Alquiler,Departamento,2,35 m²,2022-11-07T23:42:54Z
4,MLA1227217684,Alquiler Departamentos 2 Ambientes Villa Gesel...,used,7800,2023-11-06T04:01:14.000Z,ARS,False,Argentina,Bs.As. Costa Atlántica,Villa Gesell,...,-37.279080,-56.984270,1,33 m²,1,Alquiler temporal,Departamento,2,38 m²,2022-11-12T14:34:40Z
5,MLA1228990003,Departamento - Villa Crespo,used,95000,2022-12-11T20:19:56.000Z,ARS,True,Argentina,Capital Federal,Capital Federal,...,-34.601560,-58.433075,2,48 m²,2,Alquiler,Departamento,3,51 m²,2022-11-07T14:24:10Z


### Vamos a buscar como iterar en la paginación de la api

In [33]:
api

'https://api.mercadolibre.com/sites/MLA/search?category=MLA1472&_PublishedToday_YES&limit=3&offset=0'

Vamos a tener que actualizar el pedido a la api.
Para esto modificandos los parámetros de paginación offset y limit en la url

El valor máximo por solicitud para el parámetro $limit = 50$, y el valor máximo total de $offset = total/limit$



In [34]:
data['paging']

{'total': 277984, 'primary_results': 1000, 'offset': 0, 'limit': 3}

In [35]:
q = round(data['paging']['total']/50)
print(f'Para extraer toda la información a la fecha de departamentos hay que realizar {q} consultas a la api')

Para extraer toda la información a la fecha de departamentos hay que realizar 5560 consultas a la api


# Modularizamos el código

In [51]:
def ml_url(url='https://api.mercadolibre.com', method='category', product=None, limit=50, offset=0,
        today = False, **filters):
    """
    Crea una url con los parámetros necesarios para consultar la api de Mercado Libre
    """
    property_type = {
        'departamentos': 'MLA1472',
        'casas': 'MLA1466',
        'terrenos_lotes': 'MLA1493',
        'phs': 'MLA105179',
        'locales': 'MLA79242',
        'oficinas': 'MLA50538',
        'depositos_galpones': 'MLA1475',
        'cocheras': 'MLA50541',
        'campos': 'MLA1496',
        'quintas': 'MLA50547',
    }
    try:
        if product == None:
            raise ValueError(f'Se requiere id de producto, product = {product}')
        # Select method
        elif method == 'items':
            url += f'/items/{product}'
        elif product.lower() in property_type.keys():
            url += f'/sites/MLA/search?{method}={property_type[product.lower()]}'
        # Defalut
        else:
            url += f'/search?{method}={product.lower()}'

        url += '&_PublishedToday_YES' if today else ''

        if filters is not None:
            for f, v in filters.items():
                url += f'&{f}={v}'

        url += f'&limit={limit}&offset={offset}'
        return url
    except ValueError as e:
        print(f'No se pudo generar la url\n{e}')
        return ValueError

#ml_url(product='Casas', method='category', today=True, date_from="2021-01-01", date_to="2022-02-01")


In [55]:
# ['Departamentos', 'Casas', 'Phs']
ml_url(product='Phs', method='category', today =True, offset=1000, limit=3)

'https://api.mercadolibre.com/sites/MLA/search?category=MLA105179&_PublishedToday_YES&limit=3&offset=1000'

In [129]:
data['paging']['total']/limit

89850.66666666667

In [102]:
# Registros por consulta a la api.
limit = 50

# Atributos que vamos a guardar.
data_col = [
    'id', 'title', 'condition', 'price', 'stop_time', 'currency_id',
    'seller_real_estate_agency',
    'location_country_name', 'location_state_name', 'location_city_name',
    'location_neighborhood_name', 'location_latitude','location_longitude',
    ]
jsnPPcol = [
    'm_id', 'last_updated',
    ]
jsnATTcol = [
    'PROPERTY_TYPE', 'ROOMS', 'FULL_BATHROOMS',
    'BEDROOMS', 'OPERATION', 'COVERED_AREA', 'TOTAL_AREA',
    ]

# Creamos el marco de datos para guardar los registros.
df = pd.DataFrame(
    columns = data_col +  jsnPPcol + jsnATTcol
).drop(columns = ['m_id'])


prop = ['Departamentos', 'Casas', 'Phs']
for p in prop:
    # Paginación de consulta a la api.
    offset = 0    
    url = ml_url(product=p, method='category', today=True, offset=offset, limit=limit)
    r = requests.get(url)
    
    jsn, data = r.json(), json.loads(r.content)

    #_pag_tot = round(data['paging']['total']/limit) 
    
    # get = 200 # debug mode
    while  r.status_code == 200:# and _pag_tot > 0:
        # Colectamos la data de interes
        dd = pd.json_normalize(data, record_path=['results'], sep='_')[data_col]
        
        dp = pd.json_normalize(jsn['results'],
                ['prices', 'prices'], meta='id', meta_prefix='m_')[jsnPPcol]
        
        da = pd.json_normalize(jsn['results'],
                'attributes', meta='id', meta_prefix='m_') 

        da = da.loc[da.id.isin(jsnATTcol),
                    ['m_id', 'id', 'value_name']
                ].pivot(index='m_id', columns='id', values='value_name')

        da.reset_index(inplace=True)

        # Armamos el marco de datos
        _df = dd.merge(da, left_on='id', right_on='m_id', how='left')
        _df = _df.merge(dp, left_on='id', right_on='m_id', how='left')
        _df.drop(columns=['m_id_x', 'm_id_y'], inplace=True)

        # Juntamos la información
        df = pd.concat([df,_df], ignore_index=True)
        
        # Pausamos el código apara que no se enoje Juan Carlos ML.
        #time.sleep(0.5)
        
        offset += 1
        # Actualizamos url
        url = ml_url(product=p, method='category', today=True, offset=offset, limit=limit)
        r = requests.get(url)
        jsn, data = r.json(), json.loads(r.content)

        #_pag_tot -= 1

    time.sleep(10)


In [110]:
df.shape


(142650, 21)

In [111]:
t, f = df.duplicated().value_counts()
t,f

(139605, 3045)

In [112]:
df.PROPERTY_TYPE.unique()

array(['Departamento', 'Casa', 'Ph'], dtype=object)

In [109]:
df.to_csv('../Data/properties_ml.csv')

In [113]:
# Para buscar por nombre de columna
# df.iloc[:,df.columns.str.contains('av', regex=True)].columns.tolist()